# AgentCore Memory를 사용한 Strands Agent (Tool을 통한 장기 Memory)

## 소개

이 노트북은 AgentCore Memory와 Strands 프레임워크를 사용하여 **공유 장기 Memory를 가진 멀티 에이전트 시스템**을 구현하는 방법을 보여줍니다. 여러 전문화된 agent가 각 agent를 위한 전용 네임스페이스를 가진 공통 장기 Memory 저장소에 접근하면서 함께 작동하는 방법을 살펴봅니다.

### 튜토리얼 세부사항

| 정보         | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 장기 대화형                                                         |
| Agent 유형          | 여행 예약 어시스턴트                                                         |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| 튜토리얼 구성요소 | AgentCore User Preferences Memory Extraction, Memory 저장 및 검색을 위한 Tool              |
| 예제 복잡도  | 중급                                                                     |

학습 내용:

- 장기 Memory 전략을 사용한 공유 Memory 리소스 설정 방법
- 자체 Memory 네임스페이스에 접근할 수 있는 전문화된 agent 생성
- 전문화된 agent에게 위임하는 코디네이터 agent 구현
- agent 전문화를 위한 구조화된 Memory 네임스페이스 활용

## 시나리오 컨텍스트

이 예제에서는 다음으로 구성된 **여행 계획 시스템**을 만듭니다:
1. 여행 선호도와 이력에 대한 장기 Memory를 가진 항공편 예약 어시스턴트
2. 숙박 선호도에 대한 장기 Memory를 가진 호텔 예약 어시스턴트
3. 이러한 전문화된 agent를 조율하는 여행 코디네이터

각 전문화된 agent는 공통 Memory 저장소 내에서 자체 네임스페이스에 접근하여 시간이 지남에 따라 사용자 선호도에 대한 지속적인 이해를 구축할 수 있습니다. 이 접근 방식은 복잡한 도메인을 Memory 인프라를 공유하지만 자체 전문 영역을 유지하는 전문화된 agent로 분해할 수 있는 방법을 보여줍니다.

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon Bedrock AgentCore SDK

환경을 설정하고 공유 장기 Memory 리소스를 생성하여 시작하겠습니다!

## 단계 1: 환경 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작하겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
import time
from datetime import datetime
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry  # Strands agent의 hook 시스템

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("travel-assistant")

Amazon Bedrock model과 AgentCore에 대한 적절한 권한이 있는 리전을 정의합니다

In [ ]:
region = "us-west-2" # Replace with your AWS region
MODEL_ID= "global.anthropic.claude-haiku-4-5-20251001-v1:0"  # Claude Haiku 4.5 모델 사용

## 단계 2: 공유 Memory 리소스 생성
이 섹션에서는 각 agent를 위한 전용 네임스페이스를 가진 공통 장기 Memory 저장소를 생성합니다.

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "TravelAgent_LTM"  # 장기 Memory 리소스 이름
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory with Long-Term Strategy...")
    # Create the memory resource with a single long-term memory strategy
    # The {actorId} placeholder will be dynamically replaced with the actual actor ID
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Travel Agent with Long-Term Memory",
        strategies=[{
            StrategyType.USER_PREFERENCE.value: {
                "name": "UserPreferences",
                "description": "Captures user preferences",
                "namespaces": ["travel/{actorId}/preferences/"]  # {actorId}는 런타임에 실제 actor ID로 치환됨
            }
        }],
        event_expiry_days=7,  # Short-term conversation expires after 7 days
        max_wait=300,
        poll_interval=10
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 장기 Memory 전략 이해하기

우리가 생성하는 Memory 리소스는 단일 사용자 선호도 전략과 함께 AgentCore Memory의 장기 Memory 기능을 사용합니다:

1. **User Preference Memory 전략**: 대화에서 언급된 사용자 선호도를 자동으로 추출하고 통합합니다
2. **Actor 기반 네임스페이스**: 네임스페이스 경로에서 actor ID를 사용하여 각 agent를 위한 별도의 공간을 생성합니다
3. **Memory 지속성**: 만료되는 단기 Memory와 달리 추출된 선호도는 대화 만료 이후에도 지속됩니다

네임스페이스 패턴 `travel/{actorId}/preferences/`를 사용하면 각 전문화된 agent는 actor ID를 기반으로 고유한 네임스페이스를 갖게 됩니다:
- Flight agent는 다음에 접근합니다: `travel/flight-user-TIMESTAMP/preferences/`
- Hotel agent는 다음에 접근합니다: `travel/hotel-user-TIMESTAMP/preferences/`

이를 통해 각 agent는 공통 Memory 인프라를 사용하면서 자체 전문 지식을 유지할 수 있습니다.

### Agent 아이덴티티 설정

In [ ]:
# Create unique actor IDs for each specialized agent but share the session ID
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"  # 항공편 agent용 고유 actor ID
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"  # 호텔 agent용 고유 actor ID
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"  # 공유 session ID
flight_namespace = f"travel/{flight_actor_id}/preferences/"  # 항공편 agent의 memory namespace
hotel_namespace = f"travel/{hotel_actor_id}/preferences/"  # 호텔 agent의 memory namespace

In [ ]:
# Import the necessary components
from strands import Agent, tool
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider  # AgentCore Memory를 Strands tool로 제공

### 단계 3: Memory Hook Provider 생성

이 단계에서는 Memory 작업을 자동화하는 커스텀 `MemoryHookProvider` 클래스를 정의합니다. Hook은 agent의 실행 라이프사이클의 특정 지점에서 실행되는 특수 함수입니다. 우리가 생성하는 Memory hook은 하나의 주요 기능을 수행합니다:

1. **Memory 저장**: agent가 응답한 후 새로운 대화를 저장합니다

이를 통해 수동 관리 없이 원활한 Memory 경험을 제공합니다.

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient):
        self.memory_id = memory_id
        self.client = client
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2:
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                # 메시지 리스트를 역순으로 순회하여 최신 대화 추출
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Get session info from agent state
                    actor_id = event.agent.state.get("actor_id")
                    session_id = event.agent.state.get("session_id")
                    
                    if not actor_id or not session_id:
                        logger.warning("Missing actor_id or session_id in agent state")
                        return
                    
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        # agent 응답 후 자동으로 대화를 memory에 저장하도록 hook 등록
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

### Memory 접근 권한을 가진 전문화된 Agent 생성

다음으로, 전문화된 agent를 위한 시스템 프롬프트를 정의하겠습니다.

In [ ]:
# System prompt for the hotel booking specialist
HOTEL_BOOKING_PROMPT = f"""You are a hotel booking assistant. Help customers find hotels, make reservations, and answer questions about accommodations and amenities. 
Provide clear information about availability, pricing, and booking procedures in a friendly, helpful manner.Ask max two questions per turn. Keep the messages short, don't overwhelm the customer."""

# System prompt for the flight booking specialist
FLIGHT_BOOKING_PROMPT = f"""You are a flight booking assistant. Help customers find flights, make reservations, and answer questions about airlines, routes, and travel policies. 
Provide clear information about flight availability, pricing, schedules, and booking procedures in a friendly, helpful manner.Ask max two questions per turn. Keep the messages short, don't overwhelm the customer."""

### Agent Tool 구현
이제 코디네이터 agent가 사용할 수 있는 tool로 전문화된 agent를 구현하겠습니다:

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    Process and respond to flight booking queries.

    Args:
        query: A flight-related question about bookings, schedules, airlines, or travel policies

    Returns:
        Detailed flight information, booking options, or travel advice
    """
    try:
        # 항공편 agent용 memory tool provider 생성
        provider_flight = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      # Required
            actor_id=flight_actor_id, # Required
            session_id=session_id,    # Required
            region = region,
            namespace=flight_namespace  # 항공편 전용 namespace
        )
        
        flight_memory_hooks = MemoryHookProvider(memory_id, client)

        flight_agent = Agent(
            tools=provider_flight.tools,
            hooks=[flight_memory_hooks],
            model=MODEL_ID,
            system_prompt=FLIGHT_BOOKING_PROMPT,
            state={"actor_id": flight_actor_id, "session_id": session_id}  # agent state에 ID 저장
        )

        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    Process and respond to hotel booking queries.

    Args:
        query: A hotel-related question about accommodations, amenities, or reservations

    Returns:
        Detailed hotel information, booking options, or accommodation advice
    """
    try:
        # 호텔 agent용 memory tool provider 생성
        provider_hotel = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      
            actor_id=hotel_actor_id, 
            session_id=session_id,   
            region = region,
            namespace=hotel_namespace  # 호텔 전용 namespace
        )

        hotel_memory_hooks = MemoryHookProvider(memory_id, client)

        hotel_booking_agent = Agent(
            tools=provider_hotel.tools,
            hooks=[hotel_memory_hooks],
            model=MODEL_ID,
            system_prompt=HOTEL_BOOKING_PROMPT,
            state={"actor_id": hotel_actor_id, "session_id": session_id}  # agent state에 ID 저장
        )
        
        # Call the agent and return its response
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### 코디네이터 Agent 생성

마지막으로, 이러한 전문화된 tool 간을 조율하는 메인 여행 계획 agent를 생성하겠습니다:

In [ ]:
# System prompt for the coordinator agent
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a comprehensive travel planning assistant that coordinates between specialized tools:
- For flight-related queries (bookings, schedules, airlines, routes) → Use the flight_booking_assistant tool
- For hotel-related queries (accommodations, amenities, reservations) → Use the hotel_booking_assistant tool
- For complete travel packages → Use both tools as needed to provide comprehensive information
- For general travel advice or simple travel questions → Answer directly

Each agent will have its own memory in case the user asks about historic data.
When handling complex travel requests, coordinate information from both tools to create a cohesive travel plan.
Provide clear organization when presenting information from multiple sources. \
Ask max two questions per turn. Keep the messages short, don't overwhelm the customer.
"""

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]  # 전문화된 agent들을 tool로 등록
)

## 멀티 Agent Memory 시스템 테스트

여행 계획 시나리오로 멀티 에이전트 시스템을 테스트해 보겠습니다:

### 선택사항: 항공편 예약 어시스턴트의 장기 Memory 채우기
항공편 예약 agent를 채우려면 다음 셀의 주석을 해제하세요.

In [ ]:
"""flight_previous_messages = [
    ("Hi, I'm Sarah", "USER"),
    ("Hello Sarah! Welcome to FlightAssist. How can I help you with your travel plans today?", "ASSISTANT"),
    ("I'm looking to book a flight from New York to London sometime next month.", "USER"),
    ("I'd be happy to help you find flights from New York to London for next month. Do you have specific dates in mind, or are you flexible?", "ASSISTANT"),
    ("I'm thinking around the 15th to the 25th, but I can be a bit flexible.", "USER"),
    ("Great! That gives us some room to find the best options. Do you have any preferences regarding airlines or flight times?", "ASSISTANT"),
    ("I definitely prefer direct flights if possible. I really don't like layovers.", "USER"),
    ("I completely understand your preference for direct flights. Layovers can be inconvenient. Fortunately, there are several airlines offering direct flights between New York and London, including British Airways, American Airlines, Delta, and Virgin Atlantic.", "ASSISTANT"),
    ("That's good to hear. I've had good experiences with British Airways in the past.", "USER"),
    ("British Airways does offer excellent service on transatlantic routes. I'll keep that in mind when searching for options. Do you have any seating preferences or other requirements for your flight?", "ASSISTANT"),
    ("I always try to get an aisle seat. I like being able to get up without disturbing others, especially on long flights.", "USER"),
    ("An aisle seat is a great choice for long-haul flights like New York to London. I'll note your preference for aisle seating. Would you prefer to fly in the morning, afternoon, or evening?", "ASSISTANT"),
    ("I prefer overnight flights for long journeys. It helps me adjust to the time difference better.", "USER"),
    ("Overnight flights are indeed a smart choice for eastbound transatlantic travel. They allow you to arrive in London in the morning and help minimize jet lag. British Airways, Delta, and American all offer evening departures from New York that arrive in London the next morning.", "ASSISTANT"),
    ("Perfect! And I'm also wondering about baggage allowances since I'll be staying for about a week.", "USER"),
    ("For a week-long trip, most travelers find that a standard checked bag plus a carry-on is sufficient. British Airways typically allows one free checked bag on transatlantic flights in economy class, plus a carry-on and personal item. Would you like me to check the specific allowances for your preferred dates?", "ASSISTANT")
]

print("\nHydrating memories with previous conversations...")

# Save the conversation history to short-term memory
# 이전 대화 내역을 단기 memory에 저장하여 장기 memory 추출 테스트
initial = client.create_event(
    memory_id=memory_id,
    actor_id=flight_actor_id,
    session_id=session_id,
    messages=flight_previous_messages,
)
print("✓ Conversation saved in short term memory")"""

In [ ]:
travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
travel_agent("I prefer direct flights with Iberia")

In [ ]:
travel_agent("I would like a flight in the morning, in economy")

In [ ]:
travel_agent("I would like to fly from SNA, and return 15 days later")

## Memory 지속성 테스트

Memory 시스템이 올바르게 작동하는지 테스트하기 위해 여행 agent의 새 인스턴스를 생성하고 이전에 저장된 정보에 접근할 수 있는지 확인하겠습니다:

In [ ]:
time.sleep(60) # Let's give the memory some time to process the events..
# Create a new instance of the travel agent
# 새로운 agent 인스턴스를 생성하여 memory 지속성 테스트
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations
new_travel_agent("Can you remind me about flight preferences?")

## 요약

이 노트북에서 다음을 시연했습니다:

1. 여러 agent를 위한 공유 Memory 리소스를 생성하는 방법
2. Memory 접근 권한을 가진 전문화된 agent를 tool로 구현하는 방법
3. 대화 컨텍스트를 유지하면서 여러 agent 간을 조율하는 방법
4. Memory가 다른 agent 인스턴스 간에 지속되는 방법

공유 Memory를 가진 이 멀티 에이전트 아키텍처는 응집력 있는 사용자 경험을 유지하면서 전문화된 도메인을 처리할 수 있는 복잡한 대화형 AI 시스템을 구축하기 위한 강력한 접근 방식을 제공합니다.

## 정리
이 노트북에서 사용한 리소스를 정리하기 위해 Memory를 삭제하겠습니다.

In [ ]:
# client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)